# 1.데이터 탐색

In [1]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')

order = pd.read_excel('data/tables/order.xlsx')
payment = pd.read_excel('data/tables/payment.xlsx')

In [2]:
print(len(order))
order.head(3)

2070


,customer_id,order_num,date,product_num,order_count,pay_check
0,0001,20191001-0000015-01,2019-10-01 12:26:38,1,1,T
1,0002,20191001-0000026-01,2019-10-01 15:31:44,2,1,T
2,0003,20191001-0000038-01,2019-10-01 16:04:42,3,1,T


In [3]:
print(len(payment))
payment.head(3)

1184


,customer_id,date,order_price,payment_price,pay_check,payment_method,coupon,reserve_fund
0,0001,2019-10-01 12:26:38,59900,29950,T,선불금,not_used,not_used
1,0002,2019-10-01 15:31:44,59900,29950,T,신용카드,not_used,not_used
2,0003,2019-10-01 16:04:42,59900,29950,T,신용카드,not_used,not_used


---

# 2.Recency

In [4]:
pay_true = order[order['pay_check'] == 'T']

recency = pay_true.groupby(['customer_id'])['date'].max()
recency = pd.DataFrame(recency)

recency['R'] = pd.qcut(recency['date'],5, labels = np.arange(1,6))

In [5]:
recency['R'].value_counts()

5    192
1    192
4    191
3    191
2    191
Name: R, dtype: int64

In [6]:
rfm = recency.reset_index()

rfm

,customer_id,date,R
0,0001,2019-10-01 12:26:38,1
1,0002,2019-10-01 15:31:44,1
2,0003,2019-10-01 16:04:42,1
3,0005,2019-10-01 22:29:02,1
4,0006,2019-10-01 22:48:00,1
...,...,...,...
952,0990,2020-03-31 11:36:10,5
953,0991,2020-03-31 13:40:50,5
954,0992,2020-03-31 13:52:38,5
955,0993,2020-03-31 14:55:36,5


---

# 3.Frequency

In [7]:
frequency = pd.DataFrame({'customer_id':payment['customer_id'].value_counts().index, 'count':payment['customer_id'].value_counts().values})

In [8]:
frequency['count'].value_counts()

1    854
2    108
3     19
5      6
4      5
7      1
Name: count, dtype: int64

In [9]:
frequency['F'] = [5 if x >= 5 else 4 if x == 4 else 3 if x == 3 else 2 if x == 2 else 1 for x in frequency["count"]]

In [10]:
frequency.head(20)

,customer_id,count,F
0,0382,7,5
1,0313,5,5
2,0797,5,5
3,0335,5,5
4,0370,5,5
5,0044,5,5
6,0676,5,5
7,0934,4,4
8,0205,4,4
9,0193,4,4


In [11]:
rfm = pd.merge(rfm, frequency, on = 'customer_id', how = 'left').drop(['date','count'], axis = 1)
rfm

,customer_id,R,F
0,0001,1,1
1,0002,1,1
2,0003,1,1
3,0005,1,1
4,0006,1,1
...,...,...,...
952,0990,5,1
953,0991,5,1
954,0992,5,1
955,0993,5,1


---

In [12]:
payment.head(3)

,customer_id,date,order_price,payment_price,pay_check,payment_method,coupon,reserve_fund
0,0001,2019-10-01 12:26:38,59900,29950,T,선불금,not_used,not_used
1,0002,2019-10-01 15:31:44,59900,29950,T,신용카드,not_used,not_used
2,0003,2019-10-01 16:04:42,59900,29950,T,신용카드,not_used,not_used


In [13]:
g = payment.groupby('customer_id')['payment_price'].sum()

monetary = pd.DataFrame({'customer_id':g.index, 'monetary':g.values})
monetary

,customer_id,monetary
0,0001,29950
1,0002,29950
2,0003,29950
3,0004,71840
4,0005,30772
...,...,...
988,0990,29350
989,0991,111800
990,0992,59910
991,0993,41950


In [14]:
monetary['M'] = pd.qcut(monetary['monetary'],5, labels = np.arange(1,6))

In [15]:
rfm = pd.merge(rfm, monetary, on = 'customer_id', how = 'left').drop(['monetary'], axis = 1)
rfm['M'] = rfm['M'].astype('int')
rfm['R'] = rfm['R'].astype('int')

rfm['score'] = rfm['R'] + rfm['F'] + rfm['M']
rfm

,customer_id,R,F,M,score
0,0001,1,1,2,4
1,0002,1,1,2,4
2,0003,1,1,2,4
3,0005,1,1,2,4
4,0006,1,1,2,4
...,...,...,...,...,...
952,0990,5,1,2,8
953,0991,5,1,5,11
954,0992,5,1,4,10
955,0993,5,1,3,9


In [16]:
rfm_final = rfm.sort_values('score', ascending = False).reset_index(drop = 'index')
rfm_final

,customer_id,R,F,M,score
0,0313,5,5,5,15
1,0382,5,5,5,15
2,0797,5,5,5,15
3,0676,5,5,5,15
4,0629,5,4,5,14
...,...,...,...,...,...
952,0080,1,1,1,3
953,0147,1,1,1,3
954,0149,1,1,1,3
955,0152,1,1,1,3


**결과**

In [17]:
rfm_final['score'].value_counts().sort_index()

3      40
4      76
5     122
6     110
7     186
8     176
9     122
10     59
11     30
12     22
13      5
14      5
15      4
Name: score, dtype: int64

**other scoring**

In [18]:
rfm['weight_score'] = 0.3*rfm['R'] + 0.7*rfm['F'] + 1*rfm['M']

In [19]:
rfm['weight_score'].value_counts().sort_index()

2.0     40
2.3     32
2.6     38
2.7      2
2.9     28
3.0     43
3.2     40
3.3     34
3.6     37
3.7      4
3.9     44
4.0     46
4.2     32
4.3     25
4.6     30
4.7      2
4.9     38
5.0     25
5.2     41
5.3     48
5.6     59
5.7      3
5.9     33
6.0     26
6.2     29
6.3     43
6.6     26
6.7      3
6.9     21
7.0     14
7.2     11
7.3      9
7.4      4
7.6     12
7.9     14
8.0      1
8.3      5
8.6      4
8.7      1
9.0      1
9.3      2
9.7      3
10.0     4
Name: weight_score, dtype: int64

---

# 4.VIP

In [20]:
vip = rfm_final[rfm_final['score'] >= 13]
vip

,customer_id,R,F,M,score
0,0313,5,5,5,15
1,0382,5,5,5,15
2,0797,5,5,5,15
3,0676,5,5,5,15
4,0629,5,4,5,14
5,0044,4,5,5,14
6,0335,4,5,5,14
7,0370,4,5,5,14
8,0934,5,4,5,14
9,0062,5,3,5,13


In [21]:
vip_list = vip['customer_id']

**VIP 거주지역**

In [22]:
customer = pd.read_excel('data/tables/customer.xlsx')


customer[customer['customer_id'].isin(vip_list)].sort_values('address')

,customer_id,address
204,0205,경기도 김포시
61,0062,경기도 시흥시
43,0044,경기도 여주시
628,0629,경기도 용인시 기흥구
369,0370,경기도 용인시 수지구
750,0044,경기도 이천시
440,0441,경상남도 김해시
334,0335,대구광역시 동구
631,0632,부산광역시 부산진구
763,0382,서울시 강남구


**VIP 결제내역**

결제 수단 비율

In [23]:
vip_pay = payment[payment['customer_id'].isin(vip_list)]

round(vip_pay['payment_method'].value_counts()/len(vip_pay['payment_method'])*100,1)

신용카드     86.9
선불금       9.8
무통장입금     1.6
휴대폰       1.6
Name: payment_method, dtype: float64

쿠폰 사용 비율

In [24]:
round(vip_pay['coupon'].value_counts()/len(vip_pay['coupon'])*100,1)

not_used    86.9
used        13.1
Name: coupon, dtype: float64

적립금 사용 비율

In [25]:
round(vip_pay['reserve_fund'].value_counts()/len(vip_pay['reserve_fund'])*100,1)

not_used    85.2
used        14.8
Name: reserve_fund, dtype: float64

In [26]:
product = pd.read_excel('data/tables/product.xlsx')

data = pd.merge(order, product, on = 'product_num', how = 'left')
data.head(5)

,customer_id,order_num,date,product_num,order_count,pay_check,product_name,option,color,product_price
0,0001,20191001-0000015-01,2019-10-01 12:26:38,1,1,T,플러피 플리스 하이넥집업,공용,CREAM,59900
1,0001,20191001-0000015-01,2019-10-01 12:26:38,1,1,T,플러피 플리스 하이넥집업,아동,CREAM,49900
2,0002,20191001-0000026-01,2019-10-01 15:31:44,2,1,T,경량 다운 베스트,아동,PINK,59900
3,0003,20191001-0000038-01,2019-10-01 16:04:42,3,1,T,앞판그래픽 스웨트 맨투맨,공용,YELLOW,59900
4,0004,20191001-0000041-01,NaT,4,1,F,스폰지밥 후드 풀오버,공용,IVORY,39900


In [27]:
vip_ord_pro = data[data['customer_id'].isin(vip_list)]
vip_ord_pro.head(5)

,customer_id,order_num,date,product_num,order_count,pay_check,product_name,option,color,product_price
105,0044,20191010-0000045-01,2019-10-10 15:30:30,57,1,T,플러피 플리스 하이넥집업,공용,BLACK,59900
147,0062,20191014-0000054-01,2019-10-14 19:06:32,76,1,T,벨보아 레깅스,아동,CHARCOAL GREY,19900
148,0062,20191014-0000054-02,2019-10-14 19:06:32,77,1,T,기모 옆선배색 조거팬츠,아동,BLACK,29900
149,0062,20191014-0000054-03,2019-10-14 19:06:32,1,1,T,플러피 플리스 하이넥집업,공용,CREAM,59900
150,0062,20191014-0000054-03,2019-10-14 19:06:32,1,1,T,플러피 플리스 하이넥집업,아동,CREAM,49900


In [28]:
vip_ord_pro['product_name'].value_counts().head(10)

스폰지밥 전략 후드 맨투맨     8
3팩 솔리드 싱글라운드티      7
미니쭈리 5부 다이마루팬츠     6
솔리드 슬럽 라운드티셔츠      5
플러피 플리스 하이넥집업      5
그래픽형 쭈리 후드 맨투맨     4
스트라이프 싱글 라운드티      4
뒷판 전사 쭈리 후드 맨투맨    3
웰론 베스트             3
소매 그래픽 헤비쭈리 맨투맨    3
Name: product_name, dtype: int64

In [29]:
vip_ord_pro['option'].value_counts()

공용    114
여성     21
아동     16
남성     13
Name: option, dtype: int64

In [30]:
vip_ord_pro['color'].value_counts().head(10)

BLACK              32
WHITE              13
GREY               10
BLUE                8
PINK                8
NAVY                7
CHARCOAL GREY       6
TURQUOISE GREEN     6
LIGHT YELLOW        5
INDIGO              5
Name: color, dtype: int64

In [31]:
product['color'].value_counts().head(5)

BLACK            142
GREY              41
NAVY              36
WHITE             33
CHARCOAL GREY     28
Name: color, dtype: int64